In [1]:
#from diy_lib.decision_tree import Decision_tree
#from diy_lib.node import Node
import pandas
from typing import List
from itertools import combinations
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import math

In [2]:
import itertools
import numpy as np
class Node:
    """hi"""
    next_id = 0         
    def __init__(self,parent, level, data = None ,feature=None,threshold=None,gini_val=None,most_common_class=None):
        self.most_common_class=most_common_class
        self.parent = parent
        self.level = level
        self.gini_val = gini_val #none as defualt
        self.threshold = threshold #none as defualt
        self.left_child = None #none as defualt
        self.right_child = None #none as defualt
        self.feature=feature
        self.id = Node.next_id
        self.data = data
        Node.next_id += 1

    def create_children(self,lchild,rchild):
        """
        Attaches the children to the node
        """
        self.left_child = lchild
        self.right_child = rchild
    
    def get_children(self):
        """
        Returns the children of the node
        """
        return self.left_child, self.right_child
    
    def get_position(self):
        """
        Returns the position in the tree,
        and whether or not it is a root or leaf
        """
        if self.parent==None:
            return("root")
        elif self.left_child == None:
            return('leaf')
        else:
            return f"node at depth {self.level}"

    def get_info(self):
        if self.parent == None:
            parent_id = None
        else:
            parent_id = self.parent.id
        if self.get_children()[0] == None:
            children_id = [None,None]
        else:
            children_id=[self.get_children()[0].id,self.get_children()[1].id]
        return f"ID: {self.id} \nParent: {parent_id} \nChildren: {children_id}\nMost_common_class: {self.most_common_class} \nFeature: {self.feature} \nThreshold: {self.threshold} \nGini: {self.gini_val} \nDepth: {self.level} \nPosition: {self.get_position()}\nAmount of Datapoints: {len(self.data)}\nClasses of Datapoints: {Counter(self.data['type'])}"
        


def main():
    pass
if __name__=="__main__":
    main()

In [3]:
test=pandas.read_csv("../data/df_test.csv")
train=pandas.read_csv("../data/df_train.csv")
#train

In [4]:
print(len(train))

149


In [5]:
class Decision_tree:
    
    def __init__(self,depth,min_samples_split=2,entropy=False):
        self.entropy_choice=entropy
        self.depth = depth
        self.min_samples_split=min_samples_split
        self.tree = {}
    
    def _splitable(self,current_n,data):#end conditions where we dont split and instead we make leaves 
        if current_n.level >= self.depth:
            return False
        elif len(data)<self.min_samples_split:
            return False
        else:
            return True
    def _create_tree(self, current_n,tree, data):
        #should we split?
        """This function is creating nodes though recursion"""
        if self._splitable(current_n,data)==False:
            pass
        #we are splitting
        elif self.entropy == True:
            split = self.find_split(data, self.entropy) #(gini_min, threshold, best_feature)
            current_n.gini_val = split[0]
            current_n.threshold = split[1]
            current_n.feature = split[2]

            mask = data[split[2]]>split[1]
            rightsplit = data[mask]
            leftsplit = data[-mask] #check right split on feature contains higher values 
            l_child = Node(parent = current_n, data = leftsplit, gini_val = self._gini_oneside(leftsplit["type"]) ,level = current_n.level+1,most_common_class=max(Counter(leftsplit["type"]), key = Counter(leftsplit["type"]).get)) # < threshold
            self.tree[l_child.id] = l_child
            r_child = Node(parent = current_n, data = rightsplit, gini_val = self._gini_oneside(rightsplit["type"]) ,level = current_n.level+1,most_common_class=max(Counter(rightsplit["type"]), key = Counter(rightsplit["type"]).get)) # > threshold
            self.tree[r_child.id] = r_child
            current_n.create_children(l_child,r_child)
            self._create_tree(l_child,self.tree,data = leftsplit)
            self._create_tree(r_child,self.tree,data = rightsplit)
            
        else: #creates 2 children when not a leaf
            split = self.find_split(data) #(gini_min, threshold, best_feature)
            current_n.gini_val = split[0]
            current_n.threshold = split[1]
            current_n.feature = split[2]
            mask = data[split[2]]>split[1]
            rightsplit = data[mask]
            leftsplit = data[-mask] #check right split on feature contains higher values 
            l_child = Node(parent = current_n, data = leftsplit, gini_val = self._gini_oneside(leftsplit["type"]) ,level = current_n.level+1,most_common_class=max(Counter(leftsplit["type"]), key = Counter(leftsplit["type"]).get)) # < threshold
            self.tree[l_child.id] = l_child
            r_child = Node(parent = current_n, data = rightsplit, gini_val = self._gini_oneside(rightsplit["type"]) ,level = current_n.level+1,most_common_class=max(Counter(rightsplit["type"]), key = Counter(rightsplit["type"]).get)) # > threshold
            self.tree[r_child.id] = r_child
            current_n.create_children(l_child,r_child)
            self._create_tree(l_child,self.tree,data = leftsplit)
            self._create_tree(r_child,self.tree,data = rightsplit)
        return self.tree

    def _gini_oneside(self,data):
        """Takes a list of labels for one side of a 
        split and returns gini index for that side"""
        size = len(data)
        types = set(data)
        represented = Counter(data)
        gini = 1
        for key, value in represented.items():
            gini -= (value/size)**2
        return gini

    def _gini_overall(self,datal,datar):
        lsize = len(datal)
        rsize = len(datar)
        size = lsize+rsize
        return self._gini_oneside(datal)*(lsize/size)+self._gini_oneside(datar)*(rsize/size)
    
    def entropy(self, data):
        size = len(data)
        types = set(data)
        represented = Counter(data)
        entropy = 0
        for key, value in represented.items():
            entropy -= (value/size)*math.log((value/size))
        return entropy
   
    def _entropy_overall(self,datal,datar):
        lsize = len(datal)
        rsize = len(datar)
        size = lsize+rsize
        return self.entropy(datal)*(lsize/size)+self.entropy(datar)*(rsize/size)

    def fit(self,data):
        self.data = data
        Node.next_id = 0
        root = Node(parent=None, level = 0,data=self.data,most_common_class=max(Counter(data["type"]), key = Counter(data["type"]).get))
        self.tree[root.id] = root
        self.tree = self._create_tree(root,self.tree,self.data)
    
    def view(self):
        """
        Returns the tree
        """
        return self.tree
    
    def _split(self):
        #Make the split
        #calculate the threshold, for each child
        #calculate the gini value
        return #return two tuples, contaning: (feature, threshold, gini_val)

    def find_split(self,data):
        results=[]
        gini_min = 1000
        threshold = None
        best_feature= None
        if self.entropy_choice == True:
            for feature in data.loc[:, data.columns != "type"]:
                sl = list(set(data.sort_values(feature)[feature]))         #sort data by feature
                list_PS = [(sl[i+1]+sl[i])/2 for i in range(len(sl)-1)]        #find betweeen values for each pair of values in sorted 
                for PS in list_PS:#itterate through between values instead 
                    mask = train[feature]>PS
                    split = list(data[mask]["type"])
                    antisplit = list(data[-mask]["type"]);
                    entropy = self._entropy_overall(split,antisplit)
                    if entropy < gini_min:
                        gini_min = entropy
                        threshold = PS 
                        best_feature = feature          
        else:
            for feature in data.loc[:, data.columns != "type"]:
                sl = list(set(data.sort_values(feature)[feature]))         #sort data by feature
                list_PS = [(sl[i+1]+sl[i])/2 for i in range(len(sl)-1)]        #find betweeen values for each pair of values in sorted 
                for PS in list_PS:#itterate through between values instead 
                    mask = data[feature]>PS #check what data is here CHECK THIS LATER
                    split = list(data[mask]["type"])
                    antisplit = list(data[-mask]["type"]);
                    gini_val = self._gini_overall(split,antisplit)
                    if gini_val < gini_min:
                        gini_min = gini_val
                        threshold = PS 
                        best_feature = feature

                    results.append([PS, self._gini_overall(split,antisplit),feature])
        return (gini_min, threshold, best_feature)

    def predict(self, data):
        self.index_of_features=dict()
        for i,col in enumerate(data.columns):#we need this dict to index the correct value in _traverse
            self.index_of_features[col]=i
        predictions = []
        for point in data.values:
            leaf_node = self._traverse(self.tree[0],point)
            predictions.append(leaf_node.most_common_class)
        return predictions
        pass
    
    def _traverse(self,current_node,point):
        #move along decision tree until leaf is reached
        if current_node.left_child == None :
            return current_node
        else:
            threshold=current_node.threshold
            feature=current_node.feature 
            if point[self.index_of_features[feature]] >= threshold:
                return self._traverse(current_node.right_child,point)

            else:
                return self._traverse(current_node.left_child,point)

        #if leaf then return leaf
        #else find next node
        return None
        pass

In [6]:
#to do
#max depth for tree

In [44]:
T=Decision_tree(depth = 4, min_samples_split = 6) #overfit!!! gridseach
T.fit(train)

In [40]:
T.

<bound method Decision_tree.view of <__main__.Decision_tree object at 0x7faf5b33a610>>

In [36]:
def find_best(data, max_depth, min_samples_split):
    listOfTrue = []
    for d in range(max_depth):
        for s in range(2,min_samples_split):
            T=Decision_tree(depth = d, min_samples_split = s)
            T.fit(data)
            predicted = T.predict(test)
            class_ = test['type'].tolist()
            true = 0
            false = 0
            for i in range(len(predicted)):
                if class_[i] == predicted[i]:
                    true += 1
                else: 
                    false += 1
            listOfTrue.append((true/(true+false), d, s))
    return sorted(listOfTrue)

x = find_best(train, 10, 10)

print(x)

[(0.35384615384615387, 0, 2), (0.35384615384615387, 0, 3), (0.35384615384615387, 0, 4), (0.35384615384615387, 0, 5), (0.35384615384615387, 0, 6), (0.35384615384615387, 0, 7), (0.35384615384615387, 0, 8), (0.35384615384615387, 0, 9), (0.47692307692307695, 1, 2), (0.47692307692307695, 1, 3), (0.47692307692307695, 1, 4), (0.47692307692307695, 1, 5), (0.47692307692307695, 1, 6), (0.47692307692307695, 1, 7), (0.47692307692307695, 1, 8), (0.47692307692307695, 1, 9), (0.6307692307692307, 5, 2), (0.6307692307692307, 5, 3), (0.6615384615384615, 2, 2), (0.6615384615384615, 2, 3), (0.6615384615384615, 2, 4), (0.6615384615384615, 2, 5), (0.6615384615384615, 2, 6), (0.6615384615384615, 2, 7), (0.6615384615384615, 2, 8), (0.6615384615384615, 2, 9), (0.6615384615384615, 5, 4), (0.6615384615384615, 5, 5), (0.6615384615384615, 5, 6), (0.6615384615384615, 5, 7), (0.6615384615384615, 5, 8), (0.6615384615384615, 6, 2), (0.6615384615384615, 6, 3), (0.6615384615384615, 6, 7), (0.6615384615384615, 6, 8), (0.

In [39]:
#Check overfit
predicted = T.predict(test)
class_ = test['type'].tolist()

true = 0
false = 0
for i in range(len(predicted)):
    if class_[i] == predicted[i]:
        true += 1
    else: 
        false += 1
print('True predictions: ',true)
print('False predictions: ',false)
print('True: ', (true/(true+false)))

True predictions:  47
False predictions:  18
True:  0.7230769230769231


In [10]:
from sklearn.metrics import confusion_matrix

In [11]:
Counter(train["type"])

Counter({2: 53, 1: 49, 5: 9, 7: 20, 6: 6, 3: 12})

In [38]:
confusion_matrix(T.predict(test),test['type'].tolist())

array([[18,  3,  1,  0,  1,  0],
       [ 1, 15,  2,  0,  2,  0],
       [ 2,  1,  2,  0,  0,  1],
       [ 0,  1,  0,  4,  0,  0],
       [ 0,  3,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  8]])

In [13]:
confusion_matrix(T.predict(train),train['type'].tolist())

array([[49,  0,  0,  0,  0,  0],
       [ 0, 53,  0,  0,  0,  0],
       [ 0,  0, 12,  0,  0,  0],
       [ 0,  0,  0,  9,  0,  0],
       [ 0,  0,  0,  0,  6,  0],
       [ 0,  0,  0,  0,  0, 20]])

In [43]:
print(T.view()[0].get_info())

ID: 0 
Parent: None 
Children: [1, 2]
Most_common_class: 2 
Feature: Ba 
Threshold: 0.4 
Gini: 0.6199065196548418 
Depth: 0 
Position: root
Amount of Datapoints: 149
Classes of Datapoints: Counter({2: 53, 1: 49, 7: 20, 3: 12, 5: 9, 6: 6})


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=58866f15-1cff-4fef-9525-5c3070562370' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>